# Flight Punctuality Analysis at Dublin Airport

This project examines how weather conditions influence flight punctuality at Dublin Airport.  
The analysis combines flight activity data (arrivals, departures, delays, cancellations) with historical and forecast weather data from Met Éireann to identify trends, quantify the impact of adverse conditions, and project future delay probabilities. 
 
By aligning operational flight records with local weather observations, the study provides insights into how rain, wind, and visibility affect airport performance and passenger reliability.


### Notebook Control Flag Explanation

This notebook contains code to download flight history data from the Aviation Edge API.  
Because downloading six months of data can take a long time and may stress the API, we use a **control flag** called `RUN_DOWNLOAD` to decide whether the download should run.

- **RUN_DOWNLOAD = False** → The download section is skipped.  
  Use this setting when you want to run analysis, visualizations, or other notebook functionality without refreshing the data.

- **RUN_DOWNLOAD = True** → The download section executes.  
  Use this setting only when you deliberately want to refresh the flight history data and update the cumulative JSON files.

This design ensures:
- The notebook can be safely re-run without triggering unwanted downloads.
- Existing JSON files are preserved and can be loaded for analysis.
- You have full control over when heavy API calls are made.

👉 In practice: keep `RUN_DOWNLOAD = False` most of the time, and flip it to `True` only when you need new data.


In [1]:
# --- Control flag to enable/disable data refresh ---
RUN_DOWNLOAD = False   # Change to True only when you want to refresh data

### 📦 Step 2 – Install and Import Required Libraries

This step prepares the environment for the Dublin Airport Flight Rerouting Project.  
It ensures that all required Python packages are available and sets up the project’s directory structure inside the `project` root.

The notebook imports essential libraries for:

- 📊 **Data manipulation** (`pandas`, `numpy`)
- 📅 **Date and time handling** (`datetime`, `matplotlib.dates`)
- 📈 **Plotting and visualisation** (`matplotlib`, `seaborn`, `plotly`)
- 🤖 **Machine learning and model persistence** (`scikit-learn`, `joblib`)
- 📂 **File handling and paths** (`os`, `pathlib`, `json`)
- 🌐 **Web access** (`requests`)
- 🧩 **Interactivity and display** (`ipywidgets`, `IPython.display`)

It also defines key directories (`data`, `outputs`, `models`, `docs`) inside the `project` folder and ensures they exist.  
This structure keeps raw data, processed outputs, trained models, and documentation organised and reproducible.

📌 *Note: `%pip install` commands can be used inside Jupyter notebooks if a package is missing.  
For scripts or terminal use, run `pip install` directly.*


In [2]:
%pip install plotly --quiet

# --- Core Python modules ---
import json              # config files / JSON handling
import os                # operating system interactions
import time              # time management
import warnings          # manage warnings
from datetime import date, timedelta  # date calculations
from pathlib import Path              # path management
from calendar import monthrange       # leap-year safe month calculations

# --- Data science / numerical libraries ---
import numpy as np       # numerical operations
import pandas as pd      # data manipulation

# --- Plotting libraries ---
import matplotlib.pyplot as plt   # static plotting
import plotly.express as px       # interactive plotting
import seaborn as sns             # enhanced plotting

# --- Machine learning libraries ---
import joblib                     # model persistence
from sklearn.ensemble import GradientBoostingClassifier   # example model
from sklearn.linear_model import LogisticRegression       # example model
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score
)  # model evaluation
from sklearn.model_selection import (
    cross_val_score,
    train_test_split
)  # model validation

# --- API / external requests ---
import requests                   # API calls

# --- Plotting style ---
sns.set_theme(style='whitegrid')

# --- Explicit project root: programming-for-data-analytics/project ---
ROOT = Path.cwd().resolve()
if ROOT.name != "project":
    # climb up until we find project folder
    for parent in ROOT.parents:
        if parent.name == "project":
            ROOT = parent
            break

# --- Define key directories inside project ---
DATA_DIR = ROOT / "data"
OUTPUT_DIR = ROOT / "outputs"
MODEL_DIR = ROOT / "models"
DOCS_DIR = ROOT / "docs"

# --- Ensure directories exist ---
for path in [DATA_DIR, OUTPUT_DIR, MODEL_DIR, DOCS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Docs directory: {DOCS_DIR}")


Note: you may need to restart the kernel to use updated packages.
Project root: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project
Data directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\data
Output directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\outputs
Model directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\models
Docs directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\docs


### Step 3 – Utilise Helper Functions for Dublin Airport Data Processing

This section defines a set of reusable helper functions that simplify common tasks in the project.  
They are designed specifically to support the analysis of **Dublin Airport flight activity and weather data** by handling messy inputs and preparing clean datasets for exploration and modelling.

The functions help with:

- ✅ Detecting and parsing inconsistent datetime formats in flight and weather logs  
- ✅ Standardising and cleaning temperature and precipitation columns from Met Éireann datasets  
- ✅ Loading and preparing Dublin Airport daily weather data from local CSV files  
- ✅ Defining Irish seasonal boundaries (Winter, Spring, Summer, Autumn) for comparative analysis  
- ✅ Filtering weather data for a custom date range to align with flight events  
- ✅ Validating user-provided date inputs for reproducible analysis  
- ✅ Detecting header rows in raw CSV files downloaded from dashboards  

Each helper is **modular** — it performs one clear task and can be reused across notebooks and scripts.  
This improves readability, reduces duplication, and supports good programming practices for the final project.

📌 *Tip: These helpers are written to be beginner-friendly, with comments explaining their purpose and logic. They make it easier to align flight activity with weather conditions when investigating delays and cancellations.*

📖 References:  
- [Real Python – Python Helper Functions](https://realpython.com/defining-your-own-python-function/)  
- [GeeksforGeeks – Python Helper Functions](https://www.geeksforgeeks.org/python-helper-functions/)  
- [Wikipedia – DRY Principle (Don't Repeat Yourself)](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)


In [3]:
# 📂 Helper Functions for Dublin Airport Project
# These functions handle parsing dates, cleaning weather data, preparing ranges,
# defining Irish seasons, and detecting CSV headers.
# Keep them in one cell so they are easy to reuse across the notebook.

import pandas as pd
import warnings
from calendar import monthrange

# 🔍 Detect the header row in a CSV file
def detect_header(lines, keywords=("station","date","rain","temp","wind")):
    """
    Detect the most likely header row in a CSV file.
    Looks for lines containing known weather keywords and multiple columns.
    """
    for i, line in enumerate(lines):
        line_lower = line.strip().lower()
        if any(line_lower.startswith(k) for k in keywords) and "," in line:
            columns = line.split(",")
            if len(columns) > 3:  # header rows usually have multiple columns
                return i
    print("⚠️ Warning: header row not found. Defaulting to first line.")
    return 0

# 📅 Detect the most likely datetime format from sample strings
def detect_datetime_format(samples, formats, dayfirst=True, min_match_ratio=0.7, min_absolute=5):
    """
    Try each format and return the one that matches at least 70% of samples
    or at least 'min_absolute' matches. Helps ensure consistent parsing of date strings.
    """
    for fmt in formats:
        parsed = pd.to_datetime(samples, format=fmt, dayfirst=dayfirst, errors='coerce')
        matches = parsed.notna().sum()
        if matches >= max(min_absolute, int(len(samples) * min_match_ratio)):
            return fmt
    return None

# 📅 Parse a datetime column using format detection or fallback
def parse_datetime_column(df, date_col, candidate_formats=None, dayfirst=True):
    """
    Parse a datetime column using known formats.
    Falls back to flexible parsing if none match.
    """
    if candidate_formats is None:
        candidate_formats = [
            '%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M', '%d-%b-%Y %H:%M',
            '%d/%m/%Y %H:%M:%S', '%d/%m/%Y %H:%M', '%d-%m-%Y %H:%M',
            '%d %b %Y %H:%M', '%d %B %Y %H:%M',
        ]

    sample_vals = df[date_col].dropna().astype(str).head(100).tolist()
    chosen_fmt = detect_datetime_format(sample_vals, candidate_formats, dayfirst=dayfirst)

    if chosen_fmt:
        print(f"✅ Detected datetime format: {chosen_fmt}")
        return pd.to_datetime(df[date_col], format=chosen_fmt, dayfirst=dayfirst, errors='coerce')
    else:
        print("⚠️ No single format matched. Falling back to flexible parsing.")
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', message='Could not infer format')
            return pd.to_datetime(df[date_col], dayfirst=dayfirst, errors='coerce')

# 🕒 Ensure full datetime column for arrivals/departures and weather
def prepare_datetime(df, date_col='date', time_col=None):
    """
    Ensure DataFrame has a full datetime column.
    Works for datasets with combined 'date' + time or already combined datetime.
    """
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()

    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    elif date_col in df.columns and time_col:
        dt_strings = df[date_col].astype(str) + " " + df[time_col].astype(str)
        df['datetime'] = pd.to_datetime(dt_strings, format="%d-%b-%Y %H:%M", errors='coerce')
    elif date_col in df.columns:
        # Explicit format for Met Éireann hourly data: '01-jan-1945 00:00'
        df['datetime'] = pd.to_datetime(df[date_col], format="%d-%b-%Y %H:%M", errors='coerce')
    else:
        raise KeyError("No suitable date/time columns found")

    # Add convenience fields
    df['date'] = df['datetime'].dt.date
    df['hour'] = df['datetime'].dt.hour

    return df.dropna(subset=['datetime']).reset_index(drop=True)

# 🌡️ Ensure temperature column is numeric and named 'temp'
def parse_temperature_column(df, col_name='temp'):
    """
    Convert the temperature column to numeric and rename it to 'temp'.
    If no exact match, look for any column containing 'temp'.
    """
    if col_name not in df.columns:
        col_name = next((c for c in df.columns if 'temp' in c.lower()), None)
        if col_name is None:
            raise KeyError("No temperature column found.")
    if col_name != 'temp':
        df.rename(columns={col_name: 'temp'}, inplace=True)
    df['temp'] = pd.to_numeric(df['temp'], errors='coerce')
    return df

# 📂 Load cleaned weather data from local CSV
def load_cleaned_weather_data(filepath="data/dublin_airport_hourly.csv"):
    """
    Load weather dataset from CSV and strip spaces from column names.
    Default path now points to hourly Dublin Airport data.
    """
    df = pd.read_csv(filepath, low_memory=False)
    df.columns = df.columns.str.strip()
    return df

# 📊 Prepare weather data with proper datetime column
def prepare_weather_data(df, date_col='date'):
    """
    Ensure weather DataFrame has a proper datetime column.
    Works for Met Éireann hourly datasets where 'date' already includes time.
    """
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()

    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    elif date_col in df.columns:
        # Explicit format: '01-jan-1945 00:00'
        df['datetime'] = pd.to_datetime(df[date_col], format="%d-%b-%Y %H:%M", errors='coerce')
    else:
        raise ValueError("No suitable date column found in weather dataset")

    df['date'] = df['datetime'].dt.date
    df['hour'] = df['datetime'].dt.hour
    return df.dropna(subset=['datetime']).reset_index(drop=True)

# 🛠️ Clean and standardise key weather columns
def clean_weather_columns(df):
    """
    Standardise Dublin Airport hourly weather data:
    - Convert rainfall, temperature, wind speed, and pressure to numeric
    - Handle 'Tr' (trace) rainfall as 0.0
    - Ensure consistent column naming
    """
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()

    # Handle rainfall column
    if 'rain' in df.columns:
        df['rain'] = df['rain'].replace('Tr', 0.0)   # trace rainfall → 0
        df['rain'] = pd.to_numeric(df['rain'], errors='coerce')

    # Handle temperature column
    temp_col = next((c for c in df.columns if 'temp' in c), None)
    if temp_col:
        df['temp'] = pd.to_numeric(df[temp_col], errors='coerce')

    # Handle wind speed column
    if 'wdsp' in df.columns:
        df['wdsp'] = pd.to_numeric(df['wdsp'], errors='coerce')

    # Handle pressure column
    if 'msl' in df.columns:
        df['msl'] = pd.to_numeric(df['msl'], errors='coerce')

    return df

# 📆 Convert user input strings into a validated date range
def get_custom_range(start_str, end_str):
    """
    Convert string inputs into datetime objects and validate order.
    Handles ISO (YYYY-MM-DD) and European (DD/MM/YYYY) formats gracefully.
    """
    try:
        # Try ISO format first
        try:
            start = pd.to_datetime(start_str, format="%Y-%m-%d", errors="raise")
        except Exception:
            start = pd.to_datetime(start_str, dayfirst=True)

        try:
            end = pd.to_datetime(end_str, format="%Y-%m-%d %H:%M", errors="raise")
        except Exception:
            end = pd.to_datetime(end_str, dayfirst=True)

        if start > end:
            raise ValueError("Start date must be before end date.")
        return start, end
    except Exception as e:
        print(f"❌ Invalid date range: {e}")
        return None, None

import numpy as np
import pandas as pd
from calendar import monthrange

# 🍂 Define Irish seasonal boundaries for a given year (leap year safe)
def define_irish_seasons(year=2025):
    feb_days = monthrange(year, 2)[1]
    data = [
        ("Winter", pd.Timestamp(f"{year-1}-12-01"), pd.Timestamp(f"{year}-02-{feb_days} 23:59")),
        ("Spring", pd.Timestamp(f"{year}-03-01"), pd.Timestamp(f"{year}-05-31 23:59")),
        ("Summer", pd.Timestamp(f"{year}-06-01"), pd.Timestamp(f"{year}-08-31 23:59")),
        ("Autumn", pd.Timestamp(f"{year}-09-01"), pd.Timestamp(f"{year}-11-30 23:59")),
    ]
    return pd.DataFrame(data, columns=["season", "start", "end"])

# 🍂 Assign Irish seasons to a DataFrame in bulk (vectorised)
def assign_season_vectorized(df, datetime_col="datetime"):
    df = df.copy()

    # Ensure datetime dtype
    if not pd.api.types.is_datetime64_any_dtype(df[datetime_col]):
        df[datetime_col] = pd.to_datetime(df[datetime_col], errors="coerce")

    month = df[datetime_col].dt.month

    # Define conditions and choices
    conditions = [
        month.isin([12, 1, 2]),   # Winter
        month.isin([3, 4, 5]),    # Spring
        month.isin([6, 7, 8]),    # Summer
        month.isin([9, 10, 11])   # Autumn
    ]
    choices = ["Winter", "Spring", "Summer", "Autumn"]

    # Vectorised assignment
    df["season"] = np.select(conditions, choices, default="Unknown")

    # Make categorical for clarity
    df["season"] = pd.Categorical(
        df["season"],
        categories=choices + ["Unknown"],
        ordered=True
    )
    return df

### 📂 Step 4 – Download Dublin Airport Daily Data and Detect Header Row

In this step, the notebook retrieves the **Dublin Airport Daily Data CSV** directly from Met Éireann’s open data service.  
This dataset contains daily weather observations (e.g., precipitation, temperature, wind speed, radiation) recorded at Dublin Airport, which will later be aligned with flight activity logs to analyse rerouting events.

The process includes:

- 🌐 **Downloading the raw CSV** from Met Éireann using the `requests` library.  
- 📂 **Defining a local output path** (`data/dublin_airport_daily.csv`) to store the file inside the project’s `data` folder.  
- ✅ **Checking the HTTP response** to ensure the download was successful.  
- 📑 **Splitting the file into lines** so the structure can be inspected before loading into pandas.  
- 🔍 **Detecting the header row** using the `detect_header` helper function defined earlier.  
  This ensures that column names (such as `date`, `maxtp`, `mintp`, `rain`, `wdsp`) are correctly identified even if the file contains metadata lines at the top.  
- 🖨️ **Printing the detected header row** to confirm the correct starting point for parsing.

📌 *Tip: Detecting the header row is important because Met Éireann CSVs often include metadata lines before the actual data table.  
By confirming the header row, you avoid misaligned columns and ensure clean parsing in later steps.*


In [4]:
# 📂 Step 4 – Download Dublin Airport Hourly Data CSV and Detect Header Row

# Note: pathlib.Path and requests are imported earlier in the notebook, so we avoid re-importing them here.

# --- Define output path for cleaned CSV ---
DATA_PATH = Path("data/dublin_airport_hourly.csv")

# --- URL for the hourly CSV (may return 404 if file moved) ---
url = "https://cli.fusio.net/cli/climate_data/webdata/hly532.csv"

# --- Attempt to download the remote CSV, with a safe fallback to a local copy ---
try:
    response = requests.get(url, timeout=30)
except Exception as e:
    response = None
    print(f"⚠️ Network error when fetching URL: {e}")

if response is None or getattr(response, "status_code", None) != 200:
    # If remote download failed, try to use a previously saved local file if available
    if DATA_PATH.exists():
        print(f"⚠️ Remote download failed (status: {getattr(response,'status_code',None)}). Falling back to local file: {DATA_PATH}")
        text = DATA_PATH.read_text(encoding="utf-8")
        lines = text.splitlines()
    else:
        raise RuntimeError(f"❌ Failed to download data: HTTP {getattr(response,'status_code',None)} and no local fallback at {DATA_PATH}")
else:
    # Successful download — use remote content
    lines = response.text.splitlines()

# --- Detect header row using helper function ---
header_index = detect_header(lines)

# ✅ Confirm detected header row
print(f"✅ Header row detected at line {header_index}:")
print(lines[header_index])


✅ Header row detected at line 23:
date,ind,rain,ind,temp,ind,wetb,dewpt,vappr,rhum,msl,ind,wdsp,ind,wddir,ww,w,sun,vis,clht,clamt


### 📑 Step 4a – Define Delay-Relevant Weather Schema

In this step, we define a specific schema for the weather data that focuses on columns relevant to flight delays and rerouting at Dublin Airport.  
This schema will be used when loading and processing the weather dataset to ensure we only keep the necessary information for our analysis.

In [5]:
# 📑 Step 4a – Define Delay-Relevant Weather Schema

# These are the only weather variables we will keep and analyse
delay_relevant_weather_cols = [
    "datetime",      # original timestamp (needed for season assignment)
    "datetime_hour", # floored datetime for alignment with flights
    "temp",          # air temperature
    "rain",          # precipitation
    "wdsp",          # wind speed
    "wddir",         # wind direction
    "vis",           # visibility
    "clht",          # cloud height
    "ww",            # present weather code
    "w",             # past weather code
    "season"         # derived categorical label
]

print("✅ Delay-relevant weather schema defined")
print(delay_relevant_weather_cols)


✅ Delay-relevant weather schema defined
['datetime', 'datetime_hour', 'temp', 'rain', 'wdsp', 'wddir', 'vis', 'clht', 'ww', 'w', 'season']


### 📑 Step 4b – Load and Inspect Dublin Airport Hourly Weather Data

The raw hourly weather dataset from Met Éireann is provided as a CSV file with a metadata block at the top.  
Before we can clean and merge it with flight data, we need to:

1. 📥 **Download the raw CSV** directly from the Met Éireann climate data portal.  
2. 🧾 **Detect the header row** (the first line with 20+ comma‑separated fields) to skip the metadata block.  
3. 📄 **Print the metadata block** for transparency, showing station details and measurement notes.  
4. 📊 **Load the actual data into a DataFrame** (`df_weather`) using the detected header row.  
5. 🔎 **Inspect the dataset** by displaying the first few rows, checking column types, and reviewing missing values.  
6. 📈 **Generate a statistical summary** to understand ranges, distributions, and potential anomalies in the weather variables.

📌 *Why this matters:*  
This inspection step ensures that the dataset is correctly loaded, the structure is understood, and reviewers can see the raw data context before any cleaning or transformations are applied. It provides transparency and validates that the ingestion process is reproducible.


In [6]:
# 📑 Step 4b – Load and Inspect Dublin Airport Hourly Data

import requests
from io import StringIO

# --- Download raw CSV from Met Éireann ---
url = "https://cli.fusio.net/cli/climate_data/webdata/hly532.csv"
response = requests.get(url)
lines = response.text.splitlines()

# --- Detect header row: first line with 20+ comma-separated fields ---
header_index = next(i for i, line in enumerate(lines) if line.count(",") >= 20)

# --- Print metadata block (everything before header) ---
print("📄 Metadata block (before header):")
for line in lines[:header_index]:
    print(line.strip())

# --- Load CSV using detected header row ---
csv_data = "\n".join(lines[header_index:])  # header + data only
df_weather = pd.read_csv(StringIO(csv_data), low_memory=False)

# ✅ Inspect the first few rows of actual data
print("\nFirst 5 rows of Dublin Airport Hourly Data:")
display(df_weather.head())

# ✅ Check column types and missing values
print("\nDataFrame info:")
print(df_weather.info())

# ✅ Quick statistical summary
print("\nSummary statistics:")
print(df_weather.describe(include='all'))


📄 Metadata block (before header):
Station Name: DUBLIN AIRPORT
Station Height: 71 M
Latitude:53.428  ,Longitude: -6.241


date:  -  Date and Time (utc)
rain:  -  Precipitation Amount (mm)
temp:  -  Air Temperature (C)
wetb:  -  Wet Bulb Temperature (C)
dewpt: -  Dew Point Temperature (C)
rhum:  -  Relative Humidity (%)
vappr: -  Vapour Pressure (hPa)
msl:   -  Mean Sea Level Pressure (hPa)
wdsp:  -  Mean Wind Speed (knot)
wddir: -  Predominant Wind Direction (degree)
ww:    -  Synop code for Present Weather
w:     -  Synop code for Past Weather
sun:   -  Sunshine duration (hours)
vis:   -  Visibility (m)
clht:  -  Cloud height (100's of ft) - 999 if none
clamt: -  Cloud amount
ind:   -  Indicator


First 5 rows of Dublin Airport Hourly Data:


,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,01-jan-1945 00:00,2,0.0,0,4.9,0,4.6,4.4,8.2,95,...,1,0,1,0,50,4,0.0,200,2,8
1,01-jan-1945 01:00,3,0.0,0,5.1,0,4.9,4.4,8.5,97,...,1,0,1,0,45,4,0.0,200,2,8
2,01-jan-1945 02:00,2,0.0,0,5.1,0,4.8,4.4,8.5,97,...,1,0,1,0,50,4,0.0,4800,4,8
3,01-jan-1945 03:00,0,0.2,0,5.2,0,5.0,4.4,8.5,97,...,1,0,1,0,50,4,0.0,6000,4,8
4,01-jan-1945 04:00,2,0.0,0,5.6,0,5.4,5.0,8.8,97,...,1,7,1,250,50,5,0.0,6000,4,8



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708577 entries, 0 to 708576
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    708577 non-null  object 
 1   ind     708577 non-null  int64  
 2   rain    708577 non-null  float64
 3   ind.1   708577 non-null  int64  
 4   temp    708577 non-null  float64
 5   ind.2   708577 non-null  int64  
 6   wetb    708577 non-null  float64
 7   dewpt   708577 non-null  float64
 8   vappr   708577 non-null  object 
 9   rhum    708577 non-null  object 
 10  msl     708577 non-null  float64
 11  ind.3   708577 non-null  int64  
 12  wdsp    708577 non-null  int64  
 13  ind.4   708577 non-null  int64  
 14  wddir   708577 non-null  object 
 15  ww      708577 non-null  int64  
 16  w       708577 non-null  int64  
 17  sun     708577 non-null  float64
 18  vis     708577 non-null  object 
 19  clht    708577 non-null  object 
 20  clamt   708577 non-null  object

### 📑 Step 4c – Clean Dublin Airport Hourly Weather Data (Reduced Schema)

The raw hourly weather data from Met Éireann includes timestamps and meteorological variables.  
To prepare it for integration with the flight dataset, we standardise the schema and ensure  
continuous hourly coverage.

The process includes:

1. 🕒 **Parse timestamps**  
   - Convert the `date` column into a proper `datetime` object.  
   - Ensures consistent time handling across the dataset.

2. ⏱ **Create `datetime_hour`**  
   - Floor each timestamp to the nearest hour.  
   - Provides a common key for merging with flights in Step 17.

3. 🔄 **Resolve gaps in hourly data**  
   - Sort by `datetime_hour`.  
   - Apply forward and backward fill (`ffill`/`bfill`) to smooth missing observations.  
   - Maintains continuity for variables like `temp`, `rain`, `wdsp`, and `msl`.

📌 *Why this matters:*  
By cleaning and aligning weather data to hourly granularity, we ensure reproducible integration  
with the flight dataset. This step guarantees that every flight record can be matched to a  
corresponding weather observation, enabling meaningful delay vs. weather analysis.



In [7]:
# --- Parse 'date' column into proper datetime ---
df_weather['datetime'] = pd.to_datetime(
    df_weather['date'],
    format="%d-%b-%Y %H:%M",   # matches "01-jan-1945 00:00"
    errors='coerce'
)

# --- Floor to nearest hour for alignment with flights ---
df_weather['datetime_hour'] = df_weather['datetime'].dt.floor('h')

# --- Sort and forward/backward fill to resolve gaps ---
df_weather = df_weather.sort_values('datetime_hour').ffill().bfill()

# --- Restrict to schema (season will be added in Step 4d) ---
df_weather = df_weather[[c for c in delay_relevant_weather_cols if c in df_weather.columns]]

print("✅ Weather dataset cleaned and aligned to hourly granularity")
print(df_weather.head())


✅ Weather dataset cleaned and aligned to hourly granularity
             datetime       datetime_hour  temp  rain  wdsp wddir   vis clht  \
0 1945-01-01 00:00:00 1945-01-01 00:00:00   4.9   0.0     0     0   200    2   
1 1945-01-01 01:00:00 1945-01-01 01:00:00   5.1   0.0     0     0   200    2   
2 1945-01-01 02:00:00 1945-01-01 02:00:00   5.1   0.0     0     0  4800    4   
3 1945-01-01 03:00:00 1945-01-01 03:00:00   5.2   0.2     0     0  6000    4   
4 1945-01-01 04:00:00 1945-01-01 04:00:00   5.6   0.0     7   250  6000    4   

   ww  w  
0  50  4  
1  45  4  
2  50  4  
3  50  4  
4  50  5  


### 📑 Step 4d – Assign Seasons to Dublin Airport Hourly Weather Data

To enrich the weather dataset with seasonal context, we assign each record to one of the four Irish seasons.  
This provides categorical grouping for delay analysis across Winter, Spring, Summer, and Autumn.

The process includes:

1. 🍂 **Seasonal boundaries (Irish definition)**  
   - Winter: December–February (leap year safe)  
   - Spring: March–May  
   - Summer: June–August  
   - Autumn: September–November  

2. 🕒 **Vectorised assignment via helper function**  
   - Extract the month from each `datetime`.  
   - Use the `assign_season_vectorized` helper (defined in the project’s helper section) to map months to seasons.  
   - Default to `"Unknown"` if the timestamp is missing or invalid.

3. 🛠 **Categorical column for clarity**  
   - Store `season` as an ordered categorical variable (`Winter → Spring → Summer → Autumn → Unknown`).  
   - Ensures consistent grouping and reproducibility in downstream analysis.

📌 *Why this matters:*  
Adding a `season` column enables exploration of **seasonal patterns in flight delays**.  
This categorical context complements the continuous weather variables, providing richer insights  
when merged with the flight dataset in Step 17.


In [8]:
# --- Apply seasonal assignment using helper function ---
df_weather = assign_season_vectorized(df_weather, datetime_col="datetime")

# --- Ensure schema consistency ---
df_weather = df_weather[[c for c in delay_relevant_weather_cols if c in df_weather.columns]]

print("✅ Season labels assigned to weather dataset")
print(df_weather.head())


✅ Season labels assigned to weather dataset
             datetime       datetime_hour  temp  rain  wdsp wddir   vis clht  \
0 1945-01-01 00:00:00 1945-01-01 00:00:00   4.9   0.0     0     0   200    2   
1 1945-01-01 01:00:00 1945-01-01 01:00:00   5.1   0.0     0     0   200    2   
2 1945-01-01 02:00:00 1945-01-01 02:00:00   5.1   0.0     0     0  4800    4   
3 1945-01-01 03:00:00 1945-01-01 03:00:00   5.2   0.2     0     0  6000    4   
4 1945-01-01 04:00:00 1945-01-01 04:00:00   5.6   0.0     7   250  6000    4   

   ww  w  season  
0  50  4  Winter  
1  45  4  Winter  
2  50  4  Winter  
3  50  4  Winter  
4  50  5  Winter  


### 📊 Step 4e – Analyse Missing Values in Weather Dataset

After cleaning and tagging seasons, it is important to check whether the **delay‑relevant weather variables** contain any missing values.  
This ensures transparency and helps reviewers understand the reliability of the dataset before it is merged with flights.

1. 🔎 **Columns examined**  
   - `temp` (air temperature)  
   - `rain` (precipitation amount)  
   - `wdsp` (wind speed)  
   - `wddir` (wind direction)  
   - `vis` (visibility)  
   - `clht` (cloud height)  
   - `ww` (present weather code)  
   - `w` (past weather code)  
   - `season` (derived categorical label)  
   - Plus `datetime` and `datetime_hour` for alignment  

2. 📉 **Why this matters**  
   - Missing values in these fields can distort delay analysis.  
   - For example, gaps in `rain` or `vis` could hide weather events that explain flight disruptions.  
   - By checking percentages of missing values, we can decide whether to impute, forward/backward fill, or leave them as `NaN`.

3. ✅ **Outcome**  
   - If no missing values are found, the dataset is ready for merging with flights.  
   - If gaps exist, they will be documented and addressed before Step 17 (merge).

📌 *This diagnostic step ensures that only the weather variables most relevant to flight delays are validated, keeping the workflow transparent and reviewer‑friendly.*


In [9]:
# --- Restrict to schema ---
df_weather_check = df_weather[[c for c in delay_relevant_weather_cols if c in df_weather.columns]]

# --- Check for missing values ---
missing_summary_weather = df_weather_check.isna().sum().sort_values(ascending=False)
missing_percent_weather = (df_weather_check.isna().mean() * 100).round(2)
missing_columns_weather = missing_summary_weather[missing_summary_weather > 0].index.tolist()

print("🔎 Missing values per column (weather only):")
print(missing_summary_weather)
print("\n📊 Percentage of missing values per column (weather only):")
print(missing_percent_weather)
print(f"\n⚠️ Weather columns with missing values: {missing_columns_weather}")


🔎 Missing values per column (weather only):
datetime         0
datetime_hour    0
temp             0
rain             0
wdsp             0
wddir            0
vis              0
clht             0
ww               0
w                0
season           0
dtype: int64

📊 Percentage of missing values per column (weather only):
datetime         0.0
datetime_hour    0.0
temp             0.0
rain             0.0
wdsp             0.0
wddir            0.0
vis              0.0
clht             0.0
ww               0.0
w                0.0
season           0.0
dtype: float64

⚠️ Weather columns with missing values: []


### 📁 Step 5 – Save the Cleaned Dublin Airport Hourly Data CSV

After detecting the correct header row in the raw Met Éireann dataset, we now save a **cleaned version** of the Dublin Airport Hourly Data file into the project’s `data/` folder.  

This step ensures:

- 📂 The dataset is stored locally for reuse without needing to re-download from Met Éireann each time  
- 📑 All future analysis references a consistent, structured version of the data (starting at the correct header row)  
- 🔄 The workflow remains reproducible and version-controlled, supporting transparent project documentation  
- 🛠️ Analysts and reviewers can always work from the same baseline dataset, avoiding inconsistencies caused by raw file metadata  
- ⏱️ Hourly granularity is preserved, which is essential for aligning weather conditions with flight arrivals and departures  

📌 *Why this matters:*  
Saving cleaned hourly data locally is a best practice in data science. It guarantees consistency across runs, makes collaboration easier, and allows you to track changes over time.  
For Dublin Airport analysis, hourly weather data provides the necessary detail to study how conditions at specific times affect flight operations, ensuring reproducibility and transparency in your rerouting and delay modelling work.  

📖 Reference:  
- [GeeksforGeeks – Explain Data Versioning](https://www.geeksforgeeks.org/machine-learning/explain-data-versioning/)


In [10]:
# 📁 Step 5 – Save the Cleaned CSV File

# --- Ensure 'data' folder exists ---
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

# --- Save cleaned data starting from the detected header row ---
with open(DATA_PATH, "w", encoding="utf-8") as f:
    for line in lines[header_index:]:
        f.write(line + "\n")

# ✅ Confirm save location
print(f"📁 Saved cleaned climate data for Dublin Airport to: {DATA_PATH.resolve()}")


📁 Saved cleaned climate data for Dublin Airport to: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\data\dublin_airport_hourly.csv


### 📂 Step 6 – Validate Saved CSV Against Step 4 Output

Instead of re‑printing the same inspection results, this step **confirms that the locally saved CSV file is identical to the hourly dataset inspected in Step 4**.  

The process includes:

- 📂 Reloading the locally saved CSV (`data/dublin_airport_hourly.csv`)  
- 🌐 Reloading the online CSV directly from Met Éireann (skipping metadata lines)  
- ✅ Comparing the two DataFrames with `equals()` to check for exact match  
- 📊 Printing a simple confirmation message and shape comparison  

📌 *Why this matters:* This validation ensures reproducibility. It proves that the cleaned hourly file saved in Step 5 is a faithful copy of the dataset originally inspected in Step 4.  
Reviewers can trust that all downstream analysis is based on the same consistent dataset.  
For Dublin Airport analysis, this step is especially important because **hourly granularity** is required to align weather conditions with arrivals and departures at specific times.  

📖 Reference:  
- [GeeksforGeeks – Create Effective and Reproducible Code Using Pandas](https://www.geeksforgeeks.org/create-effective-and-reproducible-code-using-pandas/)


In [11]:
# 📂 Step 6 – Validate Saved CSV Against Step 4 Output (DataFrame Equality)

# --- Reload the locally saved CSV (hourly Dublin Airport data) ---
df_local = pd.read_csv("data/dublin_airport_hourly.csv", low_memory=False)

# --- Reload the online CSV (using header_index from Step 4) ---
df_online = pd.read_csv(
    "https://cli.fusio.net/cli/climate_data/webdata/hly532.csv",
    skiprows=header_index,
    low_memory=False
)

# ✅ Compare the two DataFrames for structural equality
if df_local.equals(df_online):
    print("✅ Validation successful: Local CSV matches the online dataset (structured data is consistent).")
else:
    print("❌ Validation failed: Local CSV differs from the online dataset after parsing.")

# --- Optional: show shape comparison for transparency ---
print(f"Local shape: {df_local.shape}, Online shape: {df_online.shape}")


✅ Validation successful: Local CSV matches the online dataset (structured data is consistent).
Local shape: (708577, 21), Online shape: (708577, 21)


### 📊 Step 7 – Enhance Dublin Airport Hourly Weather Data with Seasons

After validating the saved hourly dataset, this step enriches the data by preparing timestamps, cleaning numeric weather columns, and tagging each record with its Irish meteorological season.  

The process includes:

- 📂 Loading the locally saved hourly CSV (`data/dublin_airport_hourly.csv`)  
- 🕒 Preparing a full `datetime` column by combining `date` and `time`  
- 🛠️ Cleaning mixed‑type weather columns (`rain`, `temp`, `wdsp`, `msl`) into consistent numeric values  
- 🍂 Adding a `season` column using the `get_season_for_date` helper  

📌 *Why this matters:*  
By making the dataset **season‑aware**, you can easily filter and analyse weather conditions and flight delays by season. This ensures that downstream analysis captures both the **hourly granularity** and the **seasonal context**, which are critical for understanding operational impacts at Dublin Airport.


In [12]:
# 📊 Step 7 – Enhance Dublin Airport Hourly Weather Data with Seasons

# --- Load the locally saved hourly CSV ---
df_weather = load_cleaned_weather_data()

# --- Prepare datetime column (parse combined date+time) ---
df_weather = prepare_weather_data(df_weather)

# --- Clean numeric weather columns (rain, temp, wind, pressure, etc.) ---
df_weather = clean_weather_columns(df_weather)

# --- Vectorised season assignment ---
df_weather = assign_season_vectorized(df_weather, datetime_col="datetime")

# --- Restrict to unified weather schema ---
df_weather = df_weather[[c for c in delay_relevant_weather_cols if c in df_weather.columns]]

# ✅ Inspect result (all relevant columns)
print("First 10 rows with season tagging (full schema):")
display(df_weather.head(10))


First 10 rows with season tagging (full schema):


,datetime,temp,rain,wdsp,wddir,vis,clht,ww,w,season
0,1945-01-01 00:00:00,4.9,0.0,0,0,200,2,50,4,Winter
1,1945-01-01 01:00:00,5.1,0.0,0,0,200,2,45,4,Winter
2,1945-01-01 02:00:00,5.1,0.0,0,0,4800,4,50,4,Winter
3,1945-01-01 03:00:00,5.2,0.2,0,0,6000,4,50,4,Winter
4,1945-01-01 04:00:00,5.6,0.0,7,250,6000,4,50,5,Winter
5,1945-01-01 05:00:00,5.6,0.0,9,270,7000,12,51,5,Winter
6,1945-01-01 06:00:00,6.0,0.0,9,270,4000,12,51,4,Winter
7,1945-01-01 07:00:00,6.1,0.1,9,260,4000,13,50,5,Winter
8,1945-01-01 08:00:00,6.1,0.0,9,250,10000,18,20,5,Winter
9,1945-01-01 09:00:00,6.1,0.0,5,260,20000,23,2,5,Winter


### 📑 Step 8 – Download and Save Flight Activity Data

In this step, the notebook retrieves and stores **flight activity data** for Dublin Airport.  
This dataset will later be aligned with Met Éireann weather observations to analyse how conditions such as rain, wind, and visibility impact flight punctuality.

The process includes:

- 🌐 Collecting flight schedules and activity logs (arrivals, departures, delays, cancellations) from public APIs or dashboards  
- 📂 Defining a local output path (`data/dublin_airport_flights.csv`) to store the file inside the project’s `data` folder  
- ✅ Checking the response to ensure the download or export was successful  
- 📑 Parsing the raw data into a structured format, including scheduled vs actual times and delay minutes  
- 📁 Saving a cleaned version of the dataset locally for reproducibility and future analysis  

📌 *Why this matters:* Having flight activity data stored locally ensures that the project can consistently align flight events with weather conditions.  
It also supports reproducibility, version control, and enables predictive modelling of delays and cancellations without repeatedly querying external APIs.

In [13]:
# 📑 Step 8 – Download and Save Flight Activity Data
# --- Compute date range for the past six months ---
today = date.today()
six_months_ago = today - timedelta(days=182)  # approx 6 months

DATE_FROM = six_months_ago.isoformat()
DATE_TO = today.isoformat()

# --- Output directories ---
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw_flights"
RAW_DIR.mkdir(parents=True, exist_ok=True)

print(f"Date range: {DATE_FROM} to {DATE_TO}")


Date range: 2025-05-21 to 2025-11-19


### 📑 Step 9 – Dublin Airport flight information analysis 

This cell prepares the environment for **Dublin Airport flight information analysis** by defining key date ranges and output directories:

- 🗓️ **Date range:**  
  - Calculates today’s date and subtracts ~six months (182 days) to define the analysis window.  
  - Converts both dates into ISO format (`YYYY-MM-DD`) for use in API queries.  
  - These values (`DATE_FROM`, `DATE_TO`) specify the six‑month period of **flight activity data** (arrivals, departures, delays, cancellations) to be downloaded.

- 📂 **Output directories:**  
  - Creates a root `data/` folder for project storage.  
  - Inside it, a `raw_flights/` subfolder is created to hold raw JSON files retrieved from the Aviation Edge API.  
  - This ensures reproducibility and a clear separation between raw flight inputs and processed datasets.

- ✅ **Checkpoint:**  
  - Prints the computed date range so you can confirm the correct six‑month window before downloading flight information.


In [14]:
# 📑 Step 9 – Dublin Airport flight information analysis 
# --- Compute date range for the past six months ---
today = date.today()
six_months_ago = today - timedelta(days=182)  # approx 6 months

DATE_FROM = six_months_ago.isoformat()
DATE_TO = today.isoformat()

# --- Output directories ---
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw_flights"
RAW_DIR.mkdir(parents=True, exist_ok=True)

print(f"Date range: {DATE_FROM} to {DATE_TO}")


Date range: 2025-05-21 to 2025-11-19


### ✈️ Step 10 — Download Six Months of Flight History for Dublin (Arrivals and Departures)

In this step we use **Aviation Edge’s Flights History API** to collect six months of flight schedules for Dublin Airport (IATA: DUB).  
The endpoint provides detailed records for each flight, including:

- **Scheduled, estimated, and actual times** (departure and arrival)
- **Delay minutes** (either reported or inferred)
- **Flight status** (e.g., scheduled, landed, cancelled, diverted)
- **Airline and flight identifiers**

We request **both arrivals and departures** for the date range **2025‑05‑20 to 2025‑11‑18**, ensuring coverage of the most recent six months.  
The raw JSON files are saved for reproducibility in the folder:

- `data/raw_flights/dub_arrival_history.json`  
- `data/raw_flights/dub_departure_history.json`

Additionally, a `fetch_log.txt` file is generated to record progress, errors, and confirmation of successful downloads.  
This log provides transparency and makes troubleshooting easier if API requests fail or return incomplete data.

**Important notes for reproducibility:**
- The code cell was executed on **18 November 2025** using a private API key from Aviation Edge.
- To run the download yourself, you must:
  1. Sign up for an account at [aviation-edge.com](https://aviation-edge.com/) and obtain an API key.
  2. Store the key securely (e.g., as an environment variable).
  3. Set the notebook control flag `RUN_DOWNLOAD = True` to enable downloading.
- By default, the notebook will skip downloading if `RUN_DOWNLOAD = False`, and instead use the existing JSON files.  
  This prevents unnecessary API calls and ensures consistent results for reviewers.

⚠️ **Best practice:** Only re‑run the download when you want to refresh the dataset.  
Frequent downloads are unnecessary and may exceed API rate limits.

**References:**
- [Aviation Edge official site](https://aviation-edge.com/)  
- [Aviation Edge API documentation on GitHub](https://github.com/AviationEdgeAPI/Aviation-Edge-Complete-API)

In [15]:
# ✈️ Step 10 — Download Six Months of Flight History for Dublin (Arrivals and Departures)
# --- API setup ---
API_KEY = os.getenv("AVIATION_EDGE_API_KEY")   # Read API key from environment variable
if not API_KEY:
    raise RuntimeError("API key not found. Please set AVIATION_EDGE_API_KEY.")

BASE_URL = "https://aviation-edge.com/v2/public/flightsHistory"  # Endpoint for flight history
IATA_CODE = "DUB"  # Airport code for Dublin

# --- Directory setup ---
DATA_DIR = Path("data")              # Root data folder
RAW_DIR = DATA_DIR / "raw_flights"   # Subfolder for raw flight data
RAW_DIR.mkdir(parents=True, exist_ok=True)  # Create folders if missing

# --- Log file path ---
LOG_FILE = RAW_DIR / "fetch_log.txt"  # Text log for progress and errors

def log_message(message: str):
    """Print message and append to log file for tracking progress."""
    print(message)
    with open(LOG_FILE, "a", encoding="utf-8") as log:
        log.write(message + "\n")

def fetch_day(iata_code: str, flight_type: str, day: date, retries: int = 3):
    """
    Fetch flight history for a single day (arrival/departure).
    Retries up to 'retries' times if errors occur.
    """
    params = {
        "key": API_KEY,
        "code": iata_code,
        "type": flight_type,
        "date_from": day.isoformat(),
        "date_to": day.isoformat()
    }

    for attempt in range(retries):
        resp = requests.get(BASE_URL, params=params, timeout=60)  # API request
        if resp.status_code == 200:
            try:
                data = resp.json()  # Parse JSON response
                log_message(f"✅ {flight_type.capitalize()} {day}: {len(data)} records fetched")
                return data
            except Exception:
                log_message(f"⚠️ Non-JSON response on {day}: {resp.text[:200]}")
                return []
        else:
            wait = 2 ** attempt  # Exponential backoff
            log_message(f"⚠️ Error {resp.status_code} on {day} (attempt {attempt+1}/{retries}). Retrying in {wait}s...")
            time.sleep(wait)

    log_message(f"❌ Failed after {retries} retries on {day}")
    return []

def fetch_history(iata_code: str, flight_type: str, start_date: date, end_date: date):
    """
    Loop through each day in the date range and fetch daily history.
    Append results into one cumulative JSON file (avoids overwriting with empty data).
    """
    results = []
    total_days = (end_date - start_date).days + 1
    filename = RAW_DIR / f"{iata_code.lower()}_{flight_type}_history.json"

    # Load existing cumulative file if present
    if filename.exists():
        with open(filename, "r", encoding="utf-8") as f:
            try:
                results = json.load(f)
            except Exception:
                results = []
                log_message(f"⚠️ Existing {filename.name} could not be read, starting fresh.")

    # Loop through each day in range
    for i in range(total_days):
        day = start_date + timedelta(days=i)
        log_message(f"Day {i+1}/{total_days}: {day}")
        day_data = fetch_day(iata_code, flight_type, day)

        # Save only if data was fetched
        if day_data:
            results.extend(day_data)
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            log_message(f"💾 Saved {len(day_data)} records for {day} into {filename.name}")
        else:
            log_message(f"⏩ Skipped saving {day}, no data returned")

        time.sleep(1)  # Pause politely between requests

    return results

# --- Conditional download control ---
if RUN_DOWNLOAD:
    # Define date range (last ~6 months)
    today = date.today()
    start_date = today - timedelta(days=182)
    end_date = today

    log_message(f"Fetching flights from {start_date} to {end_date} for {IATA_CODE}...")

    # Fetch arrivals and departures
    arrivals = fetch_history(IATA_CODE, "arrival", start_date, end_date)
    departures = fetch_history(IATA_CODE, "departure", start_date, end_date)

    log_message(f"✅ Completed: {len(arrivals)} arrivals and {len(departures)} departures fetched.")
else:
    log_message("⏩ Skipping download step (RUN_DOWNLOAD=False). Using existing JSON files.")


⏩ Skipping download step (RUN_DOWNLOAD=False). Using existing JSON files.


### 📂 Step 11 – Inspect Headings of Downloaded JSON Files

Before tidying the flight history data, it’s important to **inspect the structure of the raw JSON files**.  
The Aviation Edge API responses can vary depending on whether the file contains arrivals or departures, and not all fields are always present.

**What this step does:**
- Loads the raw JSON files for Dublin Airport arrivals (`dub_arrival_history.json`) and departures (`dub_departure_history.json`).
- Uses `pandas.json_normalize` to flatten the nested JSON into a tabular structure.
- Prints out all available column headings so we can see which fields exist.
- Shows a sample record (truncated for readability) to preview the nested structure.

**Why this matters:**
- Helps identify which fields are consistently available and relevant for analysis.
- Prevents errors later by ensuring we only select columns that actually exist.
- Guides the design of the tidy DataFrame schema in Step 11 (e.g. keeping scheduled/actual times, delays, status, airline, etc., while dropping baggage or codeshare metadata).

👉 This inspection step is a diagnostic tool: it gives us visibility into the raw data so we can confidently build the parsing logic in the next step.


In [16]:
# 📂 Step 11 - inspect headings of downloaded JSON files
RAW_DIR = Path("data") / "raw_flights"
ARR_FILE = RAW_DIR / "dub_arrival_history.json"
DEP_FILE = RAW_DIR / "dub_departure_history.json"

def inspect_keys(json_file, sample_size=50):
    """Inspect nested keys in a JSON file by sampling records."""
    with open(json_file, "r", encoding="utf-8") as f:
        records = json.load(f)

    # Use pandas.json_normalize to flatten structure
    import pandas as pd
    df = pd.json_normalize(records)

    # Show all column headings
    print(f"\n--- Keys in {json_file.name} ---")
    print(sorted(df.columns.tolist()))

    # Optionally preview first record
    print("\nSample record:")
    print(json.dumps(records[0], indent=2)[:500])  # truncate for readability

# Inspect both files
inspect_keys(ARR_FILE)
inspect_keys(DEP_FILE)



--- Keys in dub_arrival_history.json ---
['airline.iataCode', 'airline.icaoCode', 'airline.name', 'arrival.actualRunway', 'arrival.actualTime', 'arrival.baggage', 'arrival.delay', 'arrival.estimatedRunway', 'arrival.estimatedTime', 'arrival.gate', 'arrival.iataCode', 'arrival.icaoCode', 'arrival.scheduledTime', 'arrival.terminal', 'codeshared.airline.iataCode', 'codeshared.airline.icaoCode', 'codeshared.airline.name', 'codeshared.flight.iataNumber', 'codeshared.flight.icaoNumber', 'codeshared.flight.number', 'departure.actualRunway', 'departure.actualTime', 'departure.delay', 'departure.estimatedRunway', 'departure.estimatedTime', 'departure.gate', 'departure.iataCode', 'departure.icaoCode', 'departure.scheduledTime', 'departure.terminal', 'flight.iataNumber', 'flight.icaoNumber', 'flight.number', 'status', 'type']

Sample record:
{
  "type": "arrival",
  "status": "landed",
  "departure": {
    "iataCode": "vlc",
    "icaoCode": "levc",
    "terminal": "1",
    "gate": "3",
    "dela

### 📂 Step 12 – Parse Dublin Airport Flight History JSON into Tidy DataFrames (Reduced Schema)

The raw flight history data (arrivals and departures) is stored as nested JSON.  
To make it usable for analysis and compatible with the weather dataset, we streamline  
the schema to include only the essential fields needed for delay vs. weather analysis.

The process includes:

1. 📥 **Safely extract nested fields** using a custom `_safe_get` function, avoiding `json_normalize` issues.  
2. 🕒 **Parse scheduled timestamps (`sched`) into proper datetimes** to anchor each flight record.  
3. ⏱ **Create a `datetime_hour` column floored to the hour**, ensuring alignment with the hourly weather dataset.  
4. ⏱ **Compute `delay_calc`**:  
   - Use the explicit `delay` field if available.  
   - Leave as `NaN` if no reliable delay information is present.  
5. 🛠 **Fill missing categorical fields** (`airline`, `flight_iata`) with placeholders for clarity.  
6. 🚫 **Flag cancelled flights** with an `is_cancelled` column for downstream filtering.  
7. ✈️ **Tag each record with its type** (`arrival` or `departure`) and combine into a single `df_flights` DataFrame.

📌 *Why this matters:*  
By reducing to a minimal set of columns, this step minimises missing values while  
retaining all variables necessary for analysis. The resulting tidy DataFrame is  
clean, reproducible, and ready to merge with weather data in Step 17.


In [17]:
# 📂 Step 12 – Parse Flight History JSON into Tidy DataFrames

from pathlib import Path
import json
import pandas as pd

RAW_DIR = Path("data") / "raw_flights"
ARR_FILE = RAW_DIR / "dub_arrival_history.json"
DEP_FILE = RAW_DIR / "dub_departure_history.json"

def _safe_get(d, path):
    """Get nested value from dict using dotted path, return None if missing."""
    if d is None:
        return None
    parts = path.split(".")
    val = d
    for p in parts:
        if isinstance(val, dict) and p in val:
            val = val[p]
        else:
            return None
    return val

def parse_flights(json_file, flight_type="arrival"):
    """
    Load a JSON file (arrivals or departures) and return a tidy DataFrame.
    Extract only the fields needed for the essential flight schema.
    """
    with open(json_file, "r", encoding="utf-8") as f:
        records = json.load(f)

    if not records:  # safeguard against empty files
        return pd.DataFrame()

    # Minimal mapping: only fields required for schema
    if flight_type == "arrival":
        mapping = {
            "flight_iata": "flight.iataNumber",
            "airline": "airline.name",
            "status": "status",
            "sched": "arrival.scheduledTime",
            "delay": "arrival.delay",
        }
    else:
        mapping = {
            "flight_iata": "flight.iataNumber",
            "airline": "airline.name",
            "status": "status",
            "sched": "departure.scheduledTime",
            "delay": "departure.delay",
        }

    rows = []
    for rec in records:
        row = {}
        for col, path in mapping.items():
            row[col] = _safe_get(rec, path)
        rows.append(row)

    df = pd.DataFrame(rows)

    # Parse scheduled time into datetime
    df["sched"] = pd.to_datetime(df["sched"], errors="coerce")

    # Keep both exact datetime and floored hour
    df["datetime"] = df["sched"]                # exact timestamp (same as sched in reduced schema)
    df["datetime_hour"] = df["sched"].dt.floor("h")

    # Compute delay_calc from raw delay
    df["delay_calc"] = pd.to_numeric(df["delay"], errors="coerce")

    # Fill categorical placeholders
    df["airline"] = df["airline"].fillna("Unknown")
    df["flight_iata"] = df["flight_iata"].fillna("UNK")

    # Cancellation flag
    df["is_cancelled"] = df.get("status", pd.Series(dtype="object")).astype(str).str.lower().eq("cancelled")

    # Flight type (arrival or departure)
    df["type"] = flight_type

    return df

# --- Load both arrivals and departures ---
df_arrivals = parse_flights(ARR_FILE, "arrival")
df_departures = parse_flights(DEP_FILE, "departure")

# --- Combine into one DataFrame ---
df_flights = pd.concat([df_arrivals, df_departures], ignore_index=True, sort=False)

print("Arrivals shape:", df_arrivals.shape)
print("Departures shape:", df_departures.shape)
print("Combined shape:", df_flights.shape)


Arrivals shape: (131556, 10)
Departures shape: (137720, 10)
Combined shape: (269276, 10)


### 📑 Step 12a – Define Essential Flight Schema (for Delay Analysis)

In [18]:
# 📑 Step 12a – Define Essential Flight Schema (for Delay Analysis)

essential_flight_cols = [
    "sched",         # scheduled departure/arrival time
    "datetime",      # canonical timestamp (same as sched here)
    "datetime_hour", # floored datetime for weather alignment
    "delay",         # raw delay from API
    "delay_calc",    # cleaned/calculated delay (minutes)
    "airline",       # airline name
    "flight_iata",   # flight identifier (IATA code)
    "status",        # flight status (landed, scheduled, cancelled)
    "is_cancelled",  # boolean flag for cancellations
    "type"           # flight type (arrival or departure)
]

# --- Restrict DataFrame to essential flight schema ---
df_flights = df_flights[[c for c in essential_flight_cols if c in df_flights.columns]]

print("✅ Flight dataset reduced to essential schema for delay analysis")
print(df_flights.head())


✅ Flight dataset reduced to essential schema for delay analysis
                sched            datetime       datetime_hour  delay  \
0 2025-05-20 01:00:00 2025-05-20 01:00:00 2025-05-20 01:00:00   15.0   
1 2025-05-20 01:10:00 2025-05-20 01:10:00 2025-05-20 01:00:00    NaN   
2 2025-05-20 01:15:00 2025-05-20 01:15:00 2025-05-20 01:00:00    NaN   
3 2025-05-20 04:25:00 2025-05-20 04:25:00 2025-05-20 04:00:00    NaN   
4 2025-05-20 04:25:00 2025-05-20 04:25:00 2025-05-20 04:00:00    NaN   

   delay_calc            airline flight_iata  status  is_cancelled     type  
0        15.0            ryanair      fr1739  landed         False  arrival  
1         NaN            ryanair      fr9612  landed         False  arrival  
2         NaN            ryanair       fr651  landed         False  arrival  
3         NaN  american airlines      aa8330  landed         False  arrival  
4         NaN    british airways      ba6124  landed         False  arrival  


### 📂 Step 12b – Post‑Cleaning Adjustments for Flights

After parsing the raw JSON into tidy DataFrames, a few final adjustments make the flight dataset
cleaner and more consistent for analysis:

1. 🛠 **Fill categorical placeholders**  
   - Replace missing values in `airline` with `"Unknown"` and in `flight_iata` with `"UNK"`.  
   - This ensures reviewers don’t encounter raw NaNs in key categorical fields.  
   - (Note: `terminal` is excluded from the reduced schema, so no placeholder is required.)

2. 🚫 **Handle cancelled flights**  
   - For records flagged as cancelled, set `delay_calc = 0`.  
   - This keeps the delay column numeric and avoids misleading nulls.

📌 *Why this matters:*  
These adjustments improve readability and consistency. By ensuring categorical fields are always populated
and cancelled flights have a defined delay value, the dataset becomes more reviewer‑friendly and ready
for merging with weather data in Step 17.



In [19]:
# 📂 Step 12b – Post-cleaning Adjustments for Flights (Refined)

# Fill categorical placeholders
df_flights['airline'] = df_flights['airline'].fillna("Unknown")
df_flights['flight_iata'] = df_flights['flight_iata'].fillna("UNK")

# For cancelled flights, set delay_calc = 0
df_flights.loc[df_flights['is_cancelled'], 'delay_calc'] = 0

# Keep raw 'delay' untouched for transparency
# (only drop it later if reviewers decide it's unnecessary)

print("✅ Flight dataset cleaned and aligned")
print(df_flights[['sched','datetime','datetime_hour','delay','delay_calc',
                  'airline','flight_iata','status','is_cancelled','type']].head())


✅ Flight dataset cleaned and aligned
                sched            datetime       datetime_hour  delay  \
0 2025-05-20 01:00:00 2025-05-20 01:00:00 2025-05-20 01:00:00   15.0   
1 2025-05-20 01:10:00 2025-05-20 01:10:00 2025-05-20 01:00:00    NaN   
2 2025-05-20 01:15:00 2025-05-20 01:15:00 2025-05-20 01:00:00    NaN   
3 2025-05-20 04:25:00 2025-05-20 04:25:00 2025-05-20 04:00:00    NaN   
4 2025-05-20 04:25:00 2025-05-20 04:25:00 2025-05-20 04:00:00    NaN   

   delay_calc            airline flight_iata  status  is_cancelled     type  
0        15.0            ryanair      fr1739  landed         False  arrival  
1         NaN            ryanair      fr9612  landed         False  arrival  
2         NaN            ryanair       fr651  landed         False  arrival  
3         NaN  american airlines      aa8330  landed         False  arrival  
4         NaN    british airways      ba6124  landed         False  arrival  


### 📂 Step 13 – Save Tidy Flight DataFrames

After parsing and cleaning the raw JSON flight history, we now persist the tidy DataFrames
to disk as reproducible checkpoints in the workflow.

This step performs three key actions:

1. 💾 **Save arrivals** → `dub_arrivals_tidy.csv`  
   - Contains all parsed and cleaned arrival records.

2. 💾 **Save departures** → `dub_departures_tidy.csv`  
   - Contains all parsed and cleaned departure records.

3. 💾 **Save combined dataset** → `dub_flights_tidy.csv`  
   - Concatenates arrivals and departures into a single file for holistic analysis.

📌 *Why this matters:*  
Saving tidy datasets ensures that the parsing and cleaning steps don’t need to be rerun each time.  
They serve as transparent, version‑controlled artifacts that can be shared with reviewers or reused
in later steps (e.g., merging with weather data in Step 17).  
If the `data/` folder doesn’t exist, it is created automatically, making the process robust and portable.



In [20]:
# 📂 Step 13 – Save Tidy Flight DataFrames

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)  # ensure folder exists

# Save arrivals (raw form, includes all extracted fields)
df_arrivals.to_csv(DATA_DIR / "dub_arrivals_tidy.csv", index=False)

# Save departures (raw form, includes all extracted fields)
df_departures.to_csv(DATA_DIR / "dub_departures_tidy.csv", index=False)

# Save combined dataset (already restricted to essential schema in Step 12a)
df_flights.to_csv(DATA_DIR / "dub_flights_tidy.csv", index=False)

print("✅ Saved tidy datasets into data/ folder")


✅ Saved tidy datasets into data/ folder


### 🌦️ Step 14 – Define Meteorological Seasons in Ireland for 2025

Weather analysis often benefits from grouping observations into seasonal periods.  
In Ireland, meteorological seasons are defined as:

- 🌱 **Spring** → March, April, May  
- ☀️ **Summer** → June, July, August  
- 🍂 **Autumn** → September, October, November  
- ❄️ **Winter** → December, January, February  

This step uses a helper function (`define_irish_seasons`)


In [21]:
# 🌦️ Step 14 – Define Meteorological Seasons in Ireland for 2025

# --- Generate seasonal boundaries using helper function ---
seasons_2025 = define_irish_seasons()

# --- Display formatted season ranges ---
print("📅 Irish Seasons for 2025:")
for _, row in seasons_2025.iterrows():
    print(f"  {row['season']}: {row['start'].strftime('%d-%b-%Y %H:%M')} → {row['end'].strftime('%d-%b-%Y %H:%M')}")

# --- Display the DataFrame ---    
display(seasons_2025)

📅 Irish Seasons for 2025:
  Winter: 01-Dec-2024 00:00 → 28-Feb-2025 23:59
  Spring: 01-Mar-2025 00:00 → 31-May-2025 23:59
  Summer: 01-Jun-2025 00:00 → 31-Aug-2025 23:59
  Autumn: 01-Sep-2025 00:00 → 30-Nov-2025 23:59


,season,start,end
0,Winter,2024-12-01,2025-02-28 23:59:00
1,Spring,2025-03-01,2025-05-31 23:59:00
2,Summer,2025-06-01,2025-08-31 23:59:00
3,Autumn,2025-09-01,2025-11-30 23:59:00


### 📆 Step 15 – Define and Validate a Custom Date Range (Dublin Weather)

For targeted analysis, we often need to filter weather data to a specific custom date range.  
This step validates the chosen range, checks its seasonal context, and applies it to the dataset.

Key actions performed:

1. 📅 **Define custom range**  
   - Use the helper `get_custom_range` to parse start and end dates.  
   - Example: `"2025-10-27"` → `"2025-10-31 23:59"`.

2. ⚠️ **Fallback handling**  
   - If the helper returns invalid values, default dates are applied to avoid runtime errors.

3. 🌦 **Season validation**  
   - Instead of stopping at the first match, the code now collects **all overlapping seasons**.  
   - This captures cases where the range spans multiple seasons (e.g., Autumn → Winter).  
   - The result is printed as a comma‑separated list of seasons.

4. 🛠 **Prepare weather data**  
   - Ensure a proper `datetime` column exists using `prepare_datetime`.  
   - Apply `prepare_weather_data` to clean and align the dataset.

5. 🔎 **Filter by range**  
   - Subset the Dublin Airport weather data to only include rows within the validated custom range.  
   - Display the number of rows and preview the filtered dataset.

📌 *Why this matters:*  
By validating and filtering weather data to a custom range, we create a reproducible slice of the dataset  
for focused analysis (e.g., examining delays during a specific week). Collecting all overlapping seasons  
ensures reviewers understand the full seasonal context of the chosen period, which will be critical when  
merging with flight data in Step 17.


In [22]:
# 📆 Step 15 – Define and Validate a Custom Date Range (Dublin Weather)

# --- Define custom date range using helper ---
custom_start, custom_end = get_custom_range("2025-10-27", "2025-10-31 23:59")

# --- Fallback to default if input is invalid ---
if custom_start is None or custom_end is None:
    print("⚠️ Invalid custom range returned by get_custom_range(); falling back to defaults.")
    custom_start = pd.Timestamp("2025-11-12")
    custom_end = pd.Timestamp("2025-11-16 23:59")

# --- Check which seasons the range overlaps ---
matched_seasons = []
for _, row in seasons_2025.iterrows():
    # If either start or end falls inside a season, or the season fully covers the range
    if (
        (row["start"] <= custom_start <= row["end"]) or
        (row["start"] <= custom_end <= row["end"]) or
        (custom_start <= row["start"] and custom_end >= row["end"])
    ):
        matched_seasons.append(row["season"])

# --- Display season match result ---
if matched_seasons:
    print(f"📆 The custom range overlaps with: {', '.join(matched_seasons)}")
else:
    print("⚠️ The custom range falls outside defined seasonal bounds.")

# --- Prepare filtered weather data for Dublin Airport ---
# Ensure full datetime column first
df_weather = prepare_datetime(df_weather, date_col='date', time_col='time')

# Then filter using the validated custom range
range_df = prepare_weather_data(df_weather)
range_df = range_df[(range_df['datetime'] >= custom_start) & (range_df['datetime'] <= custom_end)]

print(f"✅ Filtered Dublin weather data contains {len(range_df)} rows from {custom_start.date()} → {custom_end.date()}")
display(range_df.head())


📆 The custom range overlaps with: Autumn
✅ Filtered Dublin weather data contains 120 rows from 2025-10-27 → 2025-10-31


,datetime,temp,rain,wdsp,wddir,vis,clht,ww,w,season,date,hour
708456,2025-10-27 00:00:00,9.4,0.0,17,260,20000,27,2,11,Autumn,2025-10-27,0
708457,2025-10-27 01:00:00,9.2,0.0,17,270,20000,26,2,11,Autumn,2025-10-27,1
708458,2025-10-27 02:00:00,9.1,0.0,17,270,20000,999,2,11,Autumn,2025-10-27,2
708459,2025-10-27 03:00:00,9.4,0.0,15,270,20000,37,2,11,Autumn,2025-10-27,3
708460,2025-10-27 04:00:00,8.9,0.0,18,270,20000,999,2,11,Autumn,2025-10-27,4


### 📊 Step 15b – Reconfigure Weather Range to Match Flights

To ensure consistency between flight and weather datasets, we reconfigure the weather data  
to match the six‑month flight window (May → November 2025). This guarantees that both datasets  
cover the same period for downstream merging and analysis.

Key actions performed:

1. 🕒 **Prepare datetime column**  
   - Use `prepare_datetime` to ensure the weather dataset has a proper `datetime` field.  
   - This step standardises raw date/time strings into usable timestamps.

2. 📅 **Define six‑month custom range**  
   - Explicitly set start and end boundaries:  
     - `2025-05-20 00:00` → `2025-11-15 23:59`.  
   - Matches the flight dataset timeframe for direct comparability.

3. 🔎 **Filter weather data**  
   - Apply `prepare_weather_data` for cleaning and alignment.  
   - Subset rows to only include records within the six‑month window.  
   - Ensures only relevant weather observations are retained.

4. 📊 **Output transparency**  
   - Print the number of rows in the filtered dataset.  
   - Display the min/max dates to confirm the range.  
   - Preview the first few rows for reviewer validation.

📌 *Why this matters:*  
Aligning weather data with the flight dataset ensures analyses are based on a consistent timeframe.  
This step creates a reproducible slice of weather observations that can be merged with flight records  
in Step 17, enabling meaningful comparisons of delays against seasonal and meteorological conditions.


In [23]:
# 📊 Step 15b – Reconfigure Weather & Flight Range to Match Available Data

# Prepare weather dataset with proper datetime
range_df = prepare_weather_data(df_weather)

# Define custom range to match flights (May → Oct 2025, since weather ends 31-Oct)
custom_start = pd.Timestamp("2025-05-20 00:00")
custom_end   = pd.Timestamp("2025-10-31 23:59")

# Filter weather data to available window
range_df = range_df[(range_df['datetime'] >= custom_start) & (range_df['datetime'] <= custom_end)]

print(f"✅ Reconfigured weather dataset contains {len(range_df)} rows")
print(f"📆 Weather range: {range_df['datetime'].min().date()} → {range_df['datetime'].max().date()}")

# --- Align flights to same cutoff ---
df_flights = df_flights[(df_flights['datetime'] >= custom_start) & (df_flights['datetime'] <= custom_end)]

print(f"✈️ Flight dataset now contains {len(df_flights)} rows")
print(f"📆 Flight range: {df_flights['datetime'].min().date()} → {df_flights['datetime'].max().date()}")

display(range_df.head())
display(df_flights.head())


✅ Reconfigured weather dataset contains 3960 rows
📆 Weather range: 2025-05-20 → 2025-10-31
✈️ Flight dataset now contains 249427 rows
📆 Flight range: 2025-05-20 → 2025-10-31


,datetime,temp,rain,wdsp,wddir,vis,clht,ww,w,season,date,hour
704616,2025-05-20 00:00:00,3.9,0.0,3,110,20000,999,2,11,Spring,2025-05-20,0
704617,2025-05-20 01:00:00,3.9,0.0,3,120,20000,999,2,11,Spring,2025-05-20,1
704618,2025-05-20 02:00:00,5.0,0.0,2,110,20000,23,2,11,Spring,2025-05-20,2
704619,2025-05-20 03:00:00,4.4,0.0,2,180,20000,22,2,11,Spring,2025-05-20,3
704620,2025-05-20 04:00:00,7.0,0.0,2,200,20000,24,2,11,Spring,2025-05-20,4


,sched,datetime,datetime_hour,delay,delay_calc,airline,flight_iata,status,is_cancelled,type
0,2025-05-20 01:00:00,2025-05-20 01:00:00,2025-05-20 01:00:00,15.0,15.0,ryanair,fr1739,landed,False,arrival
1,2025-05-20 01:10:00,2025-05-20 01:10:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr9612,landed,False,arrival
2,2025-05-20 01:15:00,2025-05-20 01:15:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr651,landed,False,arrival
3,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,american airlines,aa8330,landed,False,arrival
4,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,british airways,ba6124,landed,False,arrival


### 📊 Step 16 – Load and Clean Dublin Airport Flight Data

In this step, we load the tidy flight datasets created earlier (arrivals, departures, or the combined file) and prepare them for analysis.  
The goal is to ensure that the flight records have a consistent **datetime column** and clean numeric delay values, so they can be merged with weather data later.

The process includes:

- 📂 **File resilience** → Check which tidy flight files are available in the `data/` folder  
  - Prefer the combined dataset if present, otherwise fall back to arrivals + departures.  
  - Raises a clear error if no tidy files are found.  

- 📥 **Load dataset** → Read the appropriate file(s) into memory.  
  - Harmonise columns before concatenation to ensure schema consistency.  

- 🧹 **Normalise schema** → Standardise column names to lowercase and strip whitespace.  

- 🕒 **Canonical datetime** → Create a `datetime` column based on the scheduled time (`sched`).  
  - Falls back to existing `datetime` or `date+time` if needed.  

- 🔢 **Numeric delays** → Convert delay fields into numeric values for analysis.  

- 🚫 **Final cleaning** → Drop rows without valid datetime values and reset the index.  

📌 *Why this matters:*  
This step ensures the flight dataset is reproducible, resilient, and ready for analysis.  
By anchoring on the scheduled time (`sched`), we create a consistent temporal reference point that can be reliably merged with hourly weather data in **Step 17**, enabling meaningful delay vs. weather comparisons.


In [ ]:
# 📊 Step 16 – Load and clean Dublin Airport flight data

import os
from pathlib import Path
import pandas as pd

DATA_DIR = Path("data")

# --- Inspect available tidy flight files ---
print("📂 Available files in data/:", os.listdir(DATA_DIR))

# --- Load the combined tidy flights file if present; fall back to arrivals+departures ---
flights_path = DATA_DIR / "dub_flights_tidy.csv"
arr_path = DATA_DIR / "dub_arrivals_tidy.csv"
dep_path = DATA_DIR / "dub_departures_tidy.csv"

if flights_path.exists():
    # Preferred: combined tidy dataset
    df_flights = pd.read_csv(flights_path, low_memory=False)
    source_used = "combined"
elif arr_path.exists() and dep_path.exists():
    # Fallback: arrivals + departures concatenated
    df_arrivals = pd.read_csv(arr_path, low_memory=False)
    df_departures = pd.read_csv(dep_path, low_memory=False)
    # Harmonise columns before concatenation
    common_cols = sorted(set(df_arrivals.columns).intersection(set(df_departures.columns)))
    df_flights = pd.concat(
        [df_arrivals[common_cols], df_departures[common_cols]],
        ignore_index=True
    )
    source_used = "arrivals+departures"
else:
    raise FileNotFoundError("❌ No tidy flight files found in data/. Expected dub_flights_tidy.csv or both arrivals/departures.")

print(f"✅ Loaded flight dataset from: {source_used} ({len(df_flights)} rows)")

# --- Normalise column names and whitespace ---
df_flights.columns = df_flights.columns.str.strip().str.lower()

# --- Ensure a proper datetime column ---
# Use 'sched' (scheduled time) as the canonical datetime for weather alignment
if "sched" in df_flights.columns:
    df_flights["datetime"] = pd.to_datetime(df_flights["sched"], errors="coerce")
elif "datetime" in df_flights.columns:
    df_flights["datetime"] = pd.to_datetime(df_flights["datetime"], errors="coerce")
elif "date" in df_flights.columns and "time" in df_flights.columns:
    df_flights = prepare_datetime(df_flights, date_col="date", time_col="time")
else:
    raise KeyError("❌ Flight data needs either 'sched', 'datetime', or both 'date' and 'time' columns.")

# --- Convert delay columns to numeric if present ---
for col in ["arr_delay", "dep_delay", "delay", "delay_minutes", "delay_calc"]:
    if col in df_flights.columns:
        df_flights[col] = pd.to_numeric(df_flights[col], errors="coerce")

# --- Drop rows without valid datetime and reset index ---
df_flights = df_flights.dropna(subset=["datetime"]).reset_index(drop=True)

# --- Restrict flights to weather availability (up to 31-Oct-2025) ---
cutoff_end = pd.Timestamp("2025-10-31 23:59")
df_flights = df_flights[df_flights["datetime"] <= cutoff_end].reset_index(drop=True)

print(f"🧹 Cleaned flight dataset has {len(df_flights)} rows with valid datetime")
print(f"✈️ Flight dataset aligned to weather availability: {len(df_flights)} rows")
print(f"📆 Flight range: {df_flights['datetime'].min().date()} → {df_flights['datetime'].max().date()}")

display(df_flights.head())


📂 Available files in data/: ['dublin_airport_hourly.csv', 'dub_arrivals_tidy.csv', 'dub_departures_tidy.csv', 'dub_flights_tidy.csv', 'dub_flights_weather_6months.csv', 'raw_flights']
✅ Loaded flight dataset from: combined (269276 rows)
🧹 Cleaned flight dataset has 249427 rows with valid datetime
✈️ Flight dataset aligned to weather availability: 249427 rows
📆 Flight range: 2025-05-20 → 2025-10-31


,sched,datetime,datetime_hour,delay,delay_calc,airline,flight_iata,status,is_cancelled,type
0,2025-05-20 01:00:00,2025-05-20 01:00:00,2025-05-20 01:00:00,15.0,15.0,ryanair,fr1739,landed,False,arrival
1,2025-05-20 01:10:00,2025-05-20 01:10:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr9612,landed,False,arrival
2,2025-05-20 01:15:00,2025-05-20 01:15:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr651,landed,False,arrival
3,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,american airlines,aa8330,landed,False,arrival
4,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,british airways,ba6124,landed,False,arrival


### 📊 Step 17 – Merge Flights with Weather (Floor-to-Hour Alignment)

In this step, we merge the cleaned flight dataset with the filtered Dublin Airport weather data.  
Instead of relying on `merge_asof` tolerances, we **floor both datasets to the nearest hour**.  
This ensures that every flight scheduled within a given hour is matched to the corresponding hourly weather observation.

The process includes:

- 🕒 Creating a `datetime_hour` column in both flights and weather by flooring to the hour  
- 🔗 Performing a left merge on `datetime_hour` so each flight inherits weather conditions **and seasonal labels**  
- ✅ Inspecting the merged dataset to confirm six months of flights align with six months of weather observations  

📌 *Why this matters:*  
Flooring to the hour guarantees reproducible alignment between flights and weather.  
It avoids NaN values caused by minute/second mismatches and ensures every flight has a valid weather context.  
This merged dataset forms the foundation for subsequent analyses of how meteorological conditions and seasons  
correlate with flight delays.



In [28]:
# 📊 Step 17 – Merge Flights with Weather (Unified Schema)

# --- Floor both datasets to nearest hour ---
df_flights['datetime_hour'] = df_flights['datetime'].dt.floor('h')
range_df['datetime_hour']   = range_df['datetime'].dt.floor('h')

# --- Ensure seasons are assigned before merging ---
range_df = assign_season_vectorized(range_df, datetime_col="datetime")

# --- Restrict to unified schemas ---
df_flights = df_flights[[c for c in essential_flight_cols if c in df_flights.columns]]
range_df   = range_df[[c for c in delay_relevant_weather_cols if c in range_df.columns]]

# --- Merge flights with weather schema ---
merged_df = pd.merge(
    df_flights,
    range_df,
    on="datetime_hour",
    how="left",
    validate="m:1"   # ✅ ensures each flight hour maps to at most one weather record
)

print(f"✅ Merged dataset: {len(merged_df)} flight records with hourly weather attached")

# --- Flight range with safe fallback ---
if "datetime" in merged_df.columns and merged_df["datetime"].notna().any():
    flight_min = merged_df["datetime"].min().date()
    flight_max = merged_df["datetime"].max().date()
elif "datetime_hour" in merged_df.columns and merged_df["datetime_hour"].notna().any():
    flight_min = merged_df["datetime_hour"].min().date()
    flight_max = merged_df["datetime_hour"].max().date()
else:
    raise KeyError("No valid flight timestamp column found in merged_df")

# --- Weather range ---
weather_min = range_df['datetime'].min().date()
weather_max = range_df['datetime'].max().date()

print(f"📆 Flight range:  {flight_min} → {flight_max}")
print(f"📆 Weather range: {weather_min} → {weather_max}")

# --- Explicit cutoff confirmation ---
if flight_max > weather_max:
    print(f"⚠️ Cutoff applied: Flights after {weather_max} were excluded to align with weather availability.")

# --- Match rate diagnostics ---
weather_match_rate = merged_df['datetime_hour'].notna().mean()
season_match_rate  = merged_df['season'].notna().mean()

print(f"🔎 Weather datetime match rate: {weather_match_rate:.1%}")
print(f"🔎 Season assignment rate:     {season_match_rate:.1%}")

# --- Preview merged dataset ---
display(merged_df.head())
display(merged_df.tail())


✅ Merged dataset: 249427 flight records with hourly weather attached
📆 Flight range:  2025-05-20 → 2025-10-31
📆 Weather range: 2025-05-20 → 2025-10-31
🔎 Weather datetime match rate: 100.0%
🔎 Season assignment rate:     100.0%


,sched,datetime_x,datetime_hour,delay,delay_calc,airline,flight_iata,status,is_cancelled,type,datetime_y,temp,rain,wdsp,wddir,vis,clht,ww,w,season
0,2025-05-20 01:00:00,2025-05-20 01:00:00,2025-05-20 01:00:00,15.0,15.0,ryanair,fr1739,landed,False,arrival,2025-05-20 01:00:00,3.9,0.0,3,120,20000,999,2,11,Spring
1,2025-05-20 01:10:00,2025-05-20 01:10:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr9612,landed,False,arrival,2025-05-20 01:00:00,3.9,0.0,3,120,20000,999,2,11,Spring
2,2025-05-20 01:15:00,2025-05-20 01:15:00,2025-05-20 01:00:00,NaN,NaN,ryanair,fr651,landed,False,arrival,2025-05-20 01:00:00,3.9,0.0,3,120,20000,999,2,11,Spring
3,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,american airlines,aa8330,landed,False,arrival,2025-05-20 04:00:00,7.0,0.0,2,200,20000,24,2,11,Spring
4,2025-05-20 04:25:00,2025-05-20 04:25:00,2025-05-20 04:00:00,NaN,NaN,british airways,ba6124,landed,False,arrival,2025-05-20 04:00:00,7.0,0.0,2,200,20000,24,2,11,Spring


,sched,datetime_x,datetime_hour,delay,delay_calc,airline,flight_iata,status,is_cancelled,type,datetime_y,temp,rain,wdsp,wddir,vis,clht,ww,w,season
249422,2025-10-31 19:15:00,2025-10-31 19:15:00,2025-10-31 19:00:00,NaN,NaN,british airways,ba8952,active,False,departure,2025-10-31 19:00:00,11.2,0.0,7,160,35000,130,2,11,Autumn
249423,2025-10-31 19:15:00,2025-10-31 19:15:00,2025-10-31 19:00:00,NaN,NaN,aer lingus,ei3258,active,False,departure,2025-10-31 19:00:00,11.2,0.0,7,160,35000,130,2,11,Autumn
249424,2025-10-31 19:15:00,2025-10-31 19:15:00,2025-10-31 19:00:00,NaN,NaN,british airways,ba557,active,False,departure,2025-10-31 19:00:00,11.2,0.0,7,160,35000,130,2,11,Autumn
249425,2025-10-31 19:50:00,2025-10-31 19:50:00,2025-10-31 19:00:00,31.0,31.0,transavia,hv7909,active,False,departure,2025-10-31 19:00:00,11.2,0.0,7,160,35000,130,2,11,Autumn
249426,2025-10-31 23:00:00,2025-10-31 23:00:00,2025-10-31 23:00:00,54.0,54.0,hisky,h7474,active,False,departure,2025-10-31 23:00:00,10.6,0.1,11,160,11000,45,80,82,Autumn


### 📊 Step 18 – Save Merged Flights + Weather Dataset

With flights and weather successfully merged in Step 17, we now persist the unified dataset  
to disk as a reproducible artifact. This ensures that the integration step does not need to  
be rerun each time and provides a transparent checkpoint for downstream analysis.

Key actions performed:

1. 💾 **Define output path**  
   - Save the merged dataset to `data/dub_flights_weather_6months.csv`.  
   - The filename reflects both the content (flights + weather) and the six‑month window.

2. 📥 **Write to CSV**  
   - Export the DataFrame without the index for a clean tabular structure.  
   - Ensures portability and easy re‑loading in later steps.

3. ✅ **Confirm save operation**  
   - Print the file path and row count to verify successful export.  
   - Provides reviewers with immediate feedback on dataset size and location.

📌 *Why this matters:*  
Saving the merged dataset creates a reproducible, version‑controlled artifact that can be shared  
with reviewers or reused in subsequent analyses. It marks the transition from data preparation  
to exploratory analysis, ensuring that both flight and weather records are consistently aligned  
and ready for deeper investigation.


In [29]:
# 📊 Step 18 – Save Merged Flights + Weather Dataset

OUTPUT_FILE = DATA_DIR / "dub_flights_weather_6months.csv"

# Save merged dataset to CSV
merged_df.to_csv(OUTPUT_FILE, index=False)

print(f"💾 Saved merged flights+weather dataset to {OUTPUT_FILE}")
print(f"📊 Final dataset contains {len(merged_df)} rows")


💾 Saved merged flights+weather dataset to data\dub_flights_weather_6months.csv
📊 Final dataset contains 249427 rows


### 📊 Step 19 – Audit Missing Values in Merged Dataset

After saving and reloading the merged flights + weather dataset, we now perform a  
data quality check to identify missing values across all columns. This ensures the  
dataset is complete and reliable before moving into exploratory analysis.

Key actions performed:

1. 📥 **Reload dataset**  
   - Load `dub_flights_weather_6months.csv` back into memory for validation.

2. 🔎 **Count missing values**  
   - Use `.isna().sum()` to calculate the number of missing entries per column.  
   - Sort results to highlight the most affected fields.

3. 📊 **Calculate percentages**  
   - Express missing values as percentages of total rows for each column.  
   - Provides a clearer sense of data quality impact.

4. ⚠️ **Identify problematic columns**  
   - Flag all columns with any missing values.  
   - Helps reviewers quickly see which variables may need imputation or exclusion.

📌 *Why this matters:*  
Auditing missing values is a critical step in ensuring dataset integrity.  
By quantifying both counts and percentages, reviewers can assess whether  
data gaps are minor (e.g., a few missing weather readings) or significant  
enough to affect downstream analyses of flight delays vs. weather conditions.

In [30]:
# 📊 Step 19 – Analyse Missing Values in Merged Dataset (Unified Schema)

# Load the merged dataset back from file
merged_df = pd.read_csv(DATA_DIR / "dub_flights_weather_6months.csv", low_memory=False)

# --- Combine flight + weather schemas ---
merged_schema = essential_flight_cols + delay_relevant_weather_cols

# --- Restrict to unified schema ---
merged_df = merged_df[[c for c in merged_schema if c in merged_df.columns]]

# --- Check for missing values across unified schema ---
missing_summary = merged_df.isna().sum().sort_values(ascending=False)
missing_percent = (merged_df.isna().mean() * 100).round(2)
missing_columns = missing_summary[missing_summary > 0].index.tolist()

print("🔎 Missing values per column (merged dataset):")
print(missing_summary)
print("\n📊 Percentage of missing values per column:")
print(missing_percent)
print(f"\n⚠️ Columns with missing values: {missing_columns}")


🔎 Missing values per column (merged dataset):
delay            75494
delay_calc       73692
flight_iata       1209
airline           1122
sched                0
datetime_hour        0
status               0
is_cancelled         0
type                 0
datetime_hour        0
temp                 0
rain                 0
wdsp                 0
wddir                0
vis                  0
clht                 0
ww                   0
w                    0
season               0
dtype: int64

📊 Percentage of missing values per column:
sched             0.00
datetime_hour     0.00
delay            30.27
delay_calc       29.54
airline           0.45
flight_iata       0.48
status            0.00
is_cancelled      0.00
type              0.00
datetime_hour     0.00
temp              0.00
rain              0.00
wdsp              0.00
wddir             0.00
vis               0.00
clht              0.00
ww                0.00
w                 0.00
season            0.00
dtype: float64

⚠️ Colu